In [4]:
"""Generate the code reference pages."""
import os

from pathlib import Path

import mkdocs_gen_files


nav = mkdocs_gen_files.Nav()
mod_symbol = ''#<code class="doc-symbol doc-symbol-nav doc-symbol-module"></code>'

root = Path(os.getcwd()).parent
src: Path = root

for path in sorted(src.rglob("*.py")):
    # Skip the scripts and tests directories
    if not path.relative_to(src).parts[0] == "neuro_py":
        continue

    module_path = path.relative_to(src).with_suffix("")
    doc_path = path.relative_to(src).with_suffix(".md")
    print(module_path)
    print(doc_path)
    full_doc_path = Path("reference", doc_path)
    print(full_doc_path)

    parts = tuple(module_path.parts)

    if parts[-1] == "__init__":
        parts = parts[:-1]
        doc_path = doc_path.with_name("index.md")
        full_doc_path = full_doc_path.with_name("index.md")
    elif parts[-1] == "__main__":
        continue

    nav_parts = [f"{mod_symbol} {part}" for part in parts]
    nav[tuple(nav_parts)] = doc_path.as_posix()

    # if the md file name is `module.md`, generate documentation from docstrings
    if full_doc_path.name != 'index.md':
        
        with mkdocs_gen_files.open(full_doc_path, "w") as fd:
            ident = ".".join(parts)
            fd.write(f"::: {ident}")

    else:
        this_module_path = src
        module_index = ""

        module_order = sorted(this_module_path.rglob("*.py"))
        module_order = [m.name.replace('.py', '') for m in module_order]

        for m in module_order:
            if "__init__" in m:
                continue
            if m[0] == "_":
                continue

            module_name = m
            module_index += f"* [{module_name}]" \
                            "("+m+".md)\n"

        with mkdocs_gen_files.open(full_doc_path, "w") as fd:
            fd.write(module_index)

    # with mkdocs_gen_files.open(full_doc_path, "w") as fd:
    #     identifier = ".".join(parts)
    #     print(f"---\ntitle: {identifier}\n---\n\n::: {identifier}", file=fd)

    # mkdocs_gen_files.set_edit_path(full_doc_path, path.relative_to(root))

with mkdocs_gen_files.open("reference/SUMMARY.md", "w") as nav_file:
    nav_file.writelines(nav.build_literate_nav())


neuro_py\__init__
neuro_py\__init__.md
reference\neuro_py\__init__.md


ConfigurationError: Config file 'mkdocs.yml' does not exist.

In [1]:
%reload_ext autoreload
%autoreload 2
from neuro_py.process import peri_event
import numpy as np

In [2]:

t = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
intervals = np.array([[1, 3], [4, 6], [7, 9]])
peri_event.relative_times(t, intervals)


(array([nan, 0. , 0.5, 1. , 0. , 0.5, 1. , 0. , 0.5, 1. ]),
 array([nan,  0.,  0.,  0.,  1.,  1.,  1.,  2.,  2.,  2.]))

In [20]:
from numba import jit

In [38]:
@jit(nopython=True)
def relative_times(t, intervals, values=np.array([0, 1])):    
    rt = np.zeros(len(t), dtype=np.float64) * np.nan
    intervalID = np.zeros(len(t), dtype=np.float64) * np.nan

    start_times = intervals[:, 0]
    end_times = intervals[:, 1]
    values_diff = values[1] - values[0]
    intervals_diff = end_times - start_times
    intervals_scale = values_diff / intervals_diff

    for i in range(len(t)):
        idx = np.searchsorted(start_times, t[i])
        if idx > 0 and t[i] <= end_times[idx - 1]:
            interval_i = idx - 1
        elif idx < len(start_times) and t[i] == start_times[idx]:
            interval_i = idx
        else:
            continue

        scale = intervals_scale[interval_i]
        rt[i] = ((t[i] - start_times[interval_i]) * scale) + values[0]
        intervalID[i] = interval_i

    return rt, intervalID

t = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
intervals = np.array([[1, 3], [4, 6], [7, 9]])
relative_times(t, intervals)


(array([nan, 0. , 0.5, 1. , 0. , 0.5, 1. , 0. , 0.5, 1. ]),
 array([nan,  0.,  0.,  0.,  1.,  1.,  1.,  2.,  2.,  2.]))